In [31]:
import pandas as pd
import random

# Spinetta!

In [2]:
df = pd.read_csv('data/complete_spinetta.csv', sep=';')
df.sample(5)

,disc_number,duration_ms,name,track_number,album,artist,author,lyrics_url,lyrics
222,1,205986,Yo Miro Tu Amor,4,Para Los Arboles,Luis Alberto Spinetta,Luis Alberto Spinetta,https://genius.com/Luis-alberto-spinetta-yo-mi...,"En tanto que la lluvia... ( así ) Lava, mis ..."
37,1,89773,Jingle,19,El 1er Álbum Más los Singles,Almendra,Luis Alberto Spinetta,https://genius.com/Almendra-jingle-lyrics,Vuelve del cuento feroz Su sombra está por s...
215,1,356500,Atado A Tu Frontera,9,Pan,Luis Alberto Spinetta,Luis Alberto Spinetta,https://genius.com/Luis-alberto-spinetta-atado...,En la inconsolable noche tu voz Aullando me ...
307,1,242750,Yo Quiero Ver Un Tren,2,Mondo Di Cromo,Luis Alberto Spinetta,Luis Alberto Spinetta,https://genius.com/Luis-alberto-spinetta-yo-qu...,Yo nunca me imaginé regresar a mi tiempo de ...
106,1,165893,Siguiendo Los Pasos Del Maestro,5,Los Niños Que Escriben En El Cielo,Spinetta Jade,Luis Alberto Spinetta,https://genius.com/Spinetta-jade-siguiendo-los...,-


## MARKOV

__Get list of all lyrics of non instrumental songs__

In [3]:
lyrics = df[df['lyrics'] != '-'].lyrics.values

In [4]:
def train_markov_chain(lyrics):
    """
    Args:
      - lyrics: a list of strings, where each string represents
                the lyrics of one song by an artist.
    Returns:
      A dict that maps a tuple of 2 words ("bigram") to a list of
      words that follow that bigram, representing the Markov
      chain trained on the lyrics.
    """

    # Initialize the beginning of our chain.
    chain = {
        (None, "<START>"): []
    }

    for lyric in lyrics:
        # Replace newline characters with our tag.
        lyric_newlines = lyric.replace('\n', ' <N> ')
        # Create a tuple representing the most recent (current) bigram.
        last_2 = (None, "<START>")
        for word in lyric_newlines.split():
            # Add the word as one that follows the current bigram.
            chain[last_2].append(word)
            # Shift the current bigram to account for the newly added word.
            last_2 = (last_2[1], word)
            if last_2 not in chain:
                chain[last_2] = []
        chain[last_2].append("<END>")

    return chain

In [5]:
# Train a Markov Chain over all of Logic's lyrics.
chain = train_markov_chain(lyrics)

In [6]:
def generate_new_lyrics(chain):
    """
    Args:
      - chain: a dict representing the Markov chain,
               such as one generated by generate_new_lyrics()
    Returns:
      A string representing the randomly generated song.
    """

    # a list for storing the generated words
    words = []
    # generate the first word
    word = random.choice(chain[(None, "<START>")])
    words.append(word)

    # Begin with the first bigram in our chain.
    last_2 = (None, "<START>")
    while words[-1] != "<END>":
        # Generate the next word.
        word = random.choice(chain[last_2])
        words.append(word)
        # Shift the current bigram to account for the newly added word.
        last_2 = (last_2[1], words[-1])

    # Join the words together into a string with line breaks.
    lyrics = " ".join(words[:-1])
    return "\n".join(lyrics.split("<N>"))

In [13]:
print(generate_new_lyrics(chain))

Yo Entre las miserias Y el concierto del aire 
 Y dejé mis zapatos para un niño nace 
 las nubes vendrán.... Trayendo mensajes... En el nadie de la city 
 y como un canto 
 canta porque es vida 
 al quebrar un llanto Luna y espera de serrazón Pronto esto cambiará Es un espejo de cristal


In [14]:
print(generate_new_lyrics(chain))

Gigante ¡Oh!, ¿quien te puede ver? 
 cuando la luna en diamantes... Es un milagro te haría salir Del barro no volverá Adentro queda un cuerpo Entre tus lágrimas Y el durazno partido Ya sangrando está bajo el sol a sol 
 sobre todo niños 
 si alguna vez desde mis pies Bailando porque soy mestizo Voy al norte de nada Donde sopla el viento te arrebatará... Esa hoja escrita Con tu molinete Que los amos No descansan... Ya no me digas que se inyectan ciudades ¿y qué? ¿acaso no marchitan los navíos y las rosas en los jardines nena Esta es mi feroz canción Para dormir en el cosmos Ahí va el Capitán Beto por el mal Se ríe el niño Y así yo en la canción así Una esquina desigual y un rayo al amanecer. 
 
 Tengo fiebre 
 cincuenta y pico 
 tengo un vientre blanco Dale tibia leche de tu alma Para impulsar a este mágico Y misterioso mundo Ah !, luego en el más allá Ludmila Ludmila yo veo Tus dedos de mimbre Quiero saber porque lloran... Ti, ti, ti, ti, ti, ti, ti


In [16]:
print(generate_new_lyrics(chain))

Me Caído de un vuelo limpio Entre túneles de luz Y esta vez no aceptes que otro amor vendrá. 
 
 Un sueño de alguien que se da... Solo refleja en edificios mudos... Contra el muro Al que no duermen No hay amigos, solo una Todo el mundo es extravío, 
 en sí La fuerza del fuego La voz de Cris me llega igual Y veo la cara de aquella canción sin fin Y además Dale gracias por estar cerca de ti Todo se ha movido con oscuridad Todo este tiempo Ya sé, ya no existe aquí Un congelado amanecer Tiñe de blanco hasta mi Vamos a mi! A mi! Si las flores que esperan... Condición de fulgor en algún planeta, en algún lugar 
 con las manos ante Bosnia Algo se está cayendo Es ese cielo Te voy a estar En este cementerio Qué calor hará sin vos En mi nunca la luz del amanecer... ¡Oh mi amor Esta vez solo quiero una sola cosa Dame tu luz Veo así a través de un grito De inocencia... ¿es que nunca calla Solo se ve se ama se pierde en el mar, el mar 
 Algo se acumula en mí Y la distancia no me mires dicta tu sole

In [23]:
print(generate_new_lyrics(chain))

Instant-táneas Nena, tu cabeza va a estallar Ondas en aire Ondas en aire Un columpio fabuloso en el rebaño del pastor...


In [24]:
print(generate_new_lyrics(chain))

Márcame, El mandarín Se descentró Sus manos ya no toca un libro de sal, 
 valle de eterna sal... 
 hoyo en la leña... 
 
 Un destello en el aire En la rapsodia típica de Bosnia Donde el mundo tu ser te dolerá al fin A la luna Viejo roble del camino Tus hojas siempre se la llevan detenida Como a una estrella en su luz El cielo en blanco enorme lejísimo Cielo en blanco enorme lejísimo Cielo en blanco enorme lejísimo Cielo en blanco enorme lejísimo Cielo en blanco enorme lejísimo Lejísimo lejísimo Nosotros adentro Nosotros adentro Yo quiero ver Si acaso las nubes vendrán.... Trayendo mensajes...


-----------------

# Dylan!

In [27]:
df = pd.read_csv('data/complete_dylan.csv', sep=';', header=None)
df.sample(5)

,0
286,"Change my way of thinking, bring myself a dif..."
350,Prayed in the ghetto with my face in the ceme...
234,"I married Isis on the fifth day of May, But I..."
207,"Our conversation was short and sweet, It near..."
310,"Standing on the water, casting your bread Whi..."


## MARKOV

__Get list of all lyrics of non instrumental songs__

In [30]:
lyrics = df[0].values

In [32]:
chain = train_markov_chain(lyrics)

In [33]:
print(generate_new_lyrics(chain))

Someone's You angel, you, you're as f--got me under your wing, The way you walk into the room where my pencil is at." She pulls one out now, I shall be a-changed, Ye shall be released. Yonder down here in style. By the man who was born. I saw guns and you can't make pony run, Tombstone bay dog can't get it together, Just gotta pick myself up off the picnic. I got no place I'm going to. Hey, Mr. Tambourine Man, play a song for me, no more, you can't understand. Your sons and your ghostlike soul, Who among them could ever have, Since the beginning of the roses fade And I know no one to know I love you, baby, Dream about you, baby, I know exactly where it a-hurts you, honey, My limbs are shaking, My mind weaves a symphony Of two mules, trains, and rain. I'm bound to get up in the Great North Woods, working as a parting gift. Summer days, summer nights are gone, I know I just wouldn't have a temperature? I see the geek, Who suddenly walks up to her too soon for the one that I had a job in 

In [34]:
print(generate_new_lyrics(chain))

You "There must be up to me. There's mirrors inside the gates of Eden. The savage soldier sticks his head and turned it into my ear. Madame Butterfly, she lulled me to you, but you're lazy to fetch it, And reflect from the storm." In a crowded room full of tacks, Preoccupied with his memories in a political world, Turning and a-thrashing about, As soon as the guitar played, Her mouth was watery and wet, But something is happening and you don't have to yearn for love, you don't talk so loud, Now you can feel the same, Gon' walk on your hand. There's seven breezes blowing all around my skull, I dreamed, Romantic facts of musketeers Foundationed deep somehow. Ah, but I don't care if you hear me, baby, there's smoke in your number, So sleep with one hand waving free, Silhouetted by the light of the deepest, dark forest, Where the pellets of poison are flooding their waters, Where the geese into the bells of the night is disappearing, Oh, this feeling's still here in my misery like some feu

In [35]:
print(generate_new_lyrics(chain))

If 'Twas in the bushes, blown out on the borderline, Remember me to look at, so hard to think these thoughts and never more you'll toil, You'll just lay there by the strength of strings no voice can hope to hum. The ocean wild like an oak, I've seen your lover man coming, coming 'cross the barren field, He not a friend of hers with his thumb For his tongue, it was your world. Ah, you fake just like a little girl. It was gravity which pulled us down and destiny which broke us apart, You tamed the lion lies down with the tombstone blues.


In [36]:
print(generate_new_lyrics(chain))

Well, Johnny's in the fish's mouth, Must be living in a basement down the road, you ain't got no shoes, Dad's in a blade of grass, Young man looking for someone Who's never weak, but always strong, To protect you and say a prayer for him." But, like Louise always says, "You just don't think twice, it's alright, But, if you ever hear that lonesome whistle blowing. I see the turning of the remnants of her face in the webs that they spin, Only a matter of fact, the wheels have stopped, What's good is bad, what's bad is good, you find out if she's got that I learned in Utah. Ha-va- Hava-na- Havana-gee- Havanagee-luh- Havanageeluh. Odeleyee-hoo-hoo-hoo.


In [37]:
print(generate_new_lyrics(chain))

I Senor, Senor, do you belong? Are you ready? Are you ready? Are you ready? Are you willing to engage, Saying, "We'll tie her hands and pray for rain, Got a long, hard climb. Train wheels running through my palm, the sharp hills are rising from Yellow fields with twisted oaks that groan. Won't you come in was a snare And she in between, the victim of sound, Soon shattered as a king, But, when Quinn the Eskimo gets here, them pigeons'll go to the valley and sing my song. Gonna sing it loud, sing it yourself, uh-huh.


In [38]:
print(generate_new_lyrics(chain))

How I wandered again to my knees, Don't need a woman I meet, Putting her in hope not to me, baby, I'll do anything in this telephone wire.


In [39]:
print(generate_new_lyrics(chain))

Lay, One by one they follow each other a little hilltop village they gambled for my friends, Where the sad-eyed prophets say that it's naming For the times, they are a-changing. The line, it is washed to the Lord, But you're gonna have to pay to get excited," the thief, "There's too much to argue or to judge. The bridge at midnight trembles. The country doctor rambles. Bankers' nieces seek perfection, Expecting all the women came and went, barefoot servants too. Outside in the air, Go all the while I was through, I filled up my innocence, I got until I start sinking, But I'll see you in the wood, things are falling and the Jew, "You can't repeat the past." I say, "How sweetly I sleep here alone." "The tempest may howl and the cranes fly away from myself as I am slain? Quick, Magdalena, take my life. Well, the medicine man comes and goes, My money comes and goes, my friend, is blowing in your prime, didn't you? People'd call, say, "Beware, doll, You're bound to fall." You thought they w

In [40]:
print(generate_new_lyrics(chain))

Go Oh, the flowers on your shoulder, stripes on your door? Can't you feel that sun a-shining? Groundhog running by the way she make me lonesome when you discover what you did, Don't walk on your midnight rug And your father, to your heart, Come, baby, teach me, come, baby, take me, I can't quite put my guns in the rain and the language that he was eating Wonder Bread, He ate Burger Kings, he was very bored Trying to kill him given out to find, But, if he touches you at last - oh, God! - I'm hungry as a loon/ He throws us all in a political world, Love don't have to find yourself another best friend somehow. Well, the wind, The answer is blowing in the well, the captain, he is wise, He's got a letter to me - oh, well! The hysterical bride in the pot, they're beginning to sing, What look large from a hole for me And I wait for this sweet paradise, Just remind me of my chair, said to me, go ahead and tries so hard to tell, But I'm saved, Yes, I've been told The streets of Rome are filled 

In [41]:
print(generate_new_lyrics(chain))

Come, Look out your window ledge. How long's it gonna take away my highway shoes. Well, I'm trying to get done. As the dying gunfighter lay on the avenue, Tangled up in the back. Seems like a little chicken a-down on his mind To leave his native country and his friends rearrange Their religion of the streets, child of clay, Joey, Joey, King of the night, they made a vow not to worry because, yes, they're just my foolish pride, But I can't help falling in my hand and said goodbye, ran out to the rain and the shot from a different point of view, To anyone passing by, There was no more letters, no! Not unless you mail them from Desolation Row.
